In [ ]:
import os

import cv2
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Model
import numpy as np

In [3]:
model = load_model("resnet50_2layers.h5")

In [9]:
conv_output = model.get_layer("conv5_block3_out").output
pred_ouptut = model.get_layer("dense").output
model = Model(model.input, outputs=[conv_output, pred_ouptut])

In [ ]:
for file in os.listdir(r"owoce\fake photos - bluewillow"):
    img = cv2.resize(cv2.imread(r"owoce\fake photos - bluewillow\\"+file), (192, 192))
    X = np.expand_dims(img, axis=0).astype(np.float32)
    X = preprocess_input(X)
    conv, pred = model.predict(X)

    target = np.argmax(pred, axis=1).squeeze()
    w, b = model.get_layer("dense").weights
    weights = w[:, target].numpy()
    heatmap = conv.squeeze() @ weights

    scale = 224 / 7
    plt.figure(figsize=(12, 12))
    plt.imshow(img)
    plt.imshow(zoom(heatmap, zoom=(scale, scale)), cmap="jet", alpha=0.5)
    plt.savefig(r"heatmap\\"+file)